<a href="https://colab.research.google.com/github/Zaqzxcswsde/itmo-applied-statistics/blob/main/stat_tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Imports { display-mode: "form" }

from pandas import Series, DataFrame
import pandas as pd
import numpy as np
from os import listdir
# from google.colab import drive
#drive.mount('/content/drive')
# import matplotlib.pyplot as plt
# %matplotlib inline
import itertools
from itertools import product
from scipy import special
# как можно скачать файл напрямую из гуглдиска сюда в директорию /content
# !gdown 1Xtw5-jqrDJmh318t1jtKHvOJXxMrB9XF
from math import factorial
from decimal import *
import sympy
from itertools import product
from math import fsum

# Неделя 1

In [133]:
# сколько из всех вариантов пар произведений чисел от 1 до 4 целочисленно делятся на три 
list(map(lambda a: a % 3, map(lambda a: a[0]*a[1], itertools.product(np.arange(1,5),np.arange(1,5))))).count(0)

7

In [134]:
# получить произведение всех чисел в листе
def list_product(array):
  s = 1
  for i in array:
    s *= i
  return s

In [135]:
# у скольки из всех вариантов пар произведений чисел от 1 до 4 сумма превышает произведение 
len([1 for a in itertools.product(np.arange(1,5),np.arange(1,5)) if a[0]+a[1]>a[0]*a[1]])

7

In [136]:
# формула C из комбинаторики с повторениями и без
def comb(n,k, *, repetition=0):
  if repetition: return comb(n+k-1,k)
  else: return factorial(n)/(factorial(k)*factorial(n-k))

# 4 основные функции комбинаторики объединены в одну, повторения есть? порядок важен? n? k?
def prob(n,k,o,r):
  if not o: return comb(n,k,repetition=r)
  elif r: return n**k
  else: return special.perm(n,k)

# упрощённый доступ напрямую к C из n по k
def cnk(n,k):
  return prob(n,k,0,0)

In [137]:
cnk(6,5)

6.0

In [138]:
# n = int(input('n? '))
# k = int(input('k? '))
# o = int(input('Порядок важен? 1/0 y/n ').translate(str.maketrans('yn','10')))
# r = int(input('Повторения есть? 1/0 y/n ').translate(str.maketrans('yn','10')))
n,k,o,r=[10,10,0,0]
print(prob(n,k,o,r))

1.0


In [139]:
# коккатенация листа в строчку
def conc(lst):
  c = ''
  for i in lst:
    c+=str(i)
  return c

In [140]:
# сколькими способами можно выбрать из n пар детей двих так, чтобы они были из разных пар?
pair_count=3
4*sum(range(1,pair_count))

12

In [141]:
# или так
all_possible_combinations=list(map(list,list(product(range(6),repeat=2))))
all_possible_pair_variations=[[x[0]//2,x[1]//2] for x in all_possible_combinations]
apc_with_rep_1=list(map(set,all_possible_combinations))
apc_with_rep_2=[x for x in apc_with_rep_1 if len(x)>1]
apc_without_rep=list(map(list,(set(map(tuple,apc_with_rep_2)))))
apc_with_rep_pairs_1=[[x[0]//2,x[1]//2] for x in list(map(list,apc_without_rep))]
apc_with_rep_pairs_2=list(map(set,apc_with_rep_pairs_1))
apc_without_rep_pairs=[x for x in apc_with_rep_pairs_2 if len(x)==2]
len(apc_without_rep_pairs)

12

In [142]:
# является ли число факториалом другого?
def isfactorial(x):
  if not isinstance(x,int) or x<1: return -1
  i=0
  while factorial(i)<x:
    i+=1
  if factorial(i)==x: return i
  else: return -1

# перевод числа в 6-ти-ричную систему
def tosix(n):
    if n == 0:
        return ['0']
    digits = []
    while n:
        digits.append(str(int(n % 6)))
        n //= 6
    return digits[::-1]

# является ли лист уникальным? Не работает, если там хранятся другие листы (и прочие нехешитруемые типы)
def isunique(lst):
  return len(list(lst)) == len(list(set(list(lst))))

In [143]:
# из цифр от 0 до 5 составили декартово проиведение длиной 4, в скольких вариантах нет повторений цифр?
# способ 1
len([x for x in map(set,product(range(0,6),repeat=4)) if len(x)==4])

360

In [144]:
# способ 2
sum(list(map(isunique,[''.join(tosix(x)).zfill(4) for x in range(0,6**4)])))

360

In [145]:
# или так! 6 вариантов на первую, 5 на вторую и тд до четвёртой
6*5*4*3

360

# Неделя 2

TODO: переделать парсинг, чтобы работал с дробями и переменными

## Function definitions

In [122]:
#@title Парсинг таблиц из опенеду в датафрейм/серию/лист { display-mode: "form" }
# Работает только с числовыми значениями, не работает с переменными или дробями - TODO
def OpenEduTexToArrayOfArrays(full_tex,dec=True):
  def is_float(element):
    try:
        float(element)
        return True
    except ValueError:
        return False
  full_tex=list(map(lambda a: a.split(),full_tex.split('\n')))
  for i,x in enumerate(full_tex):
    full_tex[i]=[y for y in x if is_float(y)]
  full_tex = [x for x in full_tex if len(x)>0]
  if dec:
    for i,x in enumerate(full_tex):
      full_tex[i]=list(map(lambda a: Decimal(a),x))
  return full_tex
def ArrayOfArraysWithIndexsAndColumnsToDataFrame(arrofarr):
  if len(arrofarr)<=2: raise RuntimeError('don\'t use this function for series!')
  if len(arrofarr[0])<len(arrofarr[1]): arrofarr[0] = [''] + arrofarr[0]
  npaoa = np.array(arrofarr)
  return DataFrame(npaoa[1:,1:],columns=npaoa[0,:][1:],index=npaoa[:,0][1:],dtype="object")
def TTDFOVAP(full_tex,dec=True): # TexToDataFrameOrValsAndProbs
  arrofarr=OpenEduTexToArrayOfArrays(full_tex,dec=dec)
  if len(arrofarr)==1: return arrofarr[0]
  elif len(arrofarr)==2: return (arrofarr[0],arrofarr[1])
  else: return ArrayOfArraysWithIndexsAndColumnsToDataFrame(arrofarr)

In [114]:
#@title Медиана распределения случайной величины { display-mode: "form" }
def myMedian(vals, probs):
  m_1 = [i for i,_ in enumerate(probs) if ((sum(probs[:i+1])>0.5) and (sum(probs[i:])>0.5))]
  m_1_e = [vals[v] for v in m_1]
  m_2 = m_1_e[0] if len(m_1_e)==1 else (max(m_1_e)+min(m_1_e))/2
  return m_2

In [146]:
#@title Проверка величин на независимость, !!!!!внутри DataFrame'а нужны Decimal'ы!!!!! { display-mode: "form" }
def CheckIndependence(df):
  xsi=df.sum(axis=0) # this produces series of decimals, as expected
  # eta=df.sum(axis=1) # this instead produces series of floats, not decimals - presumably pandas bug
  eta=Series({k:v.sum() for k,v in df.iterrows()})
  # xsi=xsi.apply(lambda a: Decimal(str(a)))
  # eta=eta.apply(lambda a: Decimal(str(a)))
  df2 = pd.DataFrame(columns=df.columns, index=df.index)
  for i in df.columns:
    for j in df.index:
      df2[i][j]=xsi[i]*eta[j]
  return df.equals(df2)

In [115]:
#@title Математическое ожидание распределения случайной величины { display-mode: "form" }
def myExpectedValue(vals, probs):
  vals = np.asarray(vals)
  probs = np.asarray(probs)
  return (vals * probs).sum()

In [116]:
#@title Дана таблица совместного распределения случайных величин -> получаем мат.ожидание произведения случайных величин { display-mode: "form" }
def TablToExpectedOfMult(df):
  df2=DataFrame(columns=df.columns,index=df.index)
  for i in df2.columns:
    for j in df2.index:
      df2[i][j]=i*j
  ss={}
  for i in df2.columns:
    for j in df2.index:
      if df2[i][j] in ss.keys():
        ss[df2[i][j]]+=df[i][j]
      else:
        ss[df2[i][j]]=df[i][j]
  return sum(np.asarray(list(ss.keys()))*np.asarray(list(ss.values())))

In [118]:
#@title Дано распределение случайной величины -> вернуть дисперсию { display-mode: "form" }
def MyVariance(vals,probs):
  ev = myExpectedValue(vals,probs)
  vals2=[(x-ev)**2 for x in vals]
  disp = myExpectedValue(vals2,probs)
  return disp

In [119]:
#@title Дано распределение случайной величины -> вернуть дисперсию 2 { display-mode: "form" }
def MyVariance2(vals,probs):
  ev = myExpectedValue(vals,probs)
  vals2=[x**2 for x in vals]
  disp = myExpectedValue(vals2,probs)
  return disp-(ev)**2

In [155]:
#@title Дана таблица совместного распределения случайных величин -> получаем мат.ожидание обеих случайных величин - сначала по столбцам, потом по строкам { display-mode: "form" }
def TablToExpectedOfBoth(df):
  xsi,eta=MyJointDistributionToMarginalDistribution(df)
  xsi_var=myExpectedValue(xsi.index,xsi.values)
  eta_var=myExpectedValue(eta.index,eta.values)
  return (xsi_var,eta_var)

In [120]:
#@title Совметное распределение в маргинальное распределение { display-mode: "form" }
def MyJointDistributionToMarginalDistribution(df):
  xsi=df.sum(axis=0)
  eta=Series({k:v.sum() for k,v in df.iterrows()})
  return (xsi,eta)

In [121]:
#@title Получить СКО из маргинального распределения или из дисперсии { display-mode: "form" }
def MyStandardDeviation(vals,probs,dev=None):
  if not dev: return MyVariance(vals,probs)**(1/2)
  else: return dev**(1/2)

---


## Code itself

In [123]:
# Количество орлов/решек при трёх подбрасываниях монеты
list(map(lambda a: a.count(0), itertools.product([0,1],repeat=3)))
list(map(lambda a: a.count(1), itertools.product([0,1],repeat=3)))

[0, 1, 1, 2, 1, 2, 2, 3]

In [124]:
# Проверка независимости
# arr=np.array([[0.02,0.02,0.06,0.1],[0.03,0.03,0.09,0.15],[0.02,0.02,0.06,0.1],[0.03,0.03,0.09,0.15]])
# arr2 = np.vectorize(lambda a: Decimal(str(a)))(arr)
# df = pd.DataFrame(arr2,columns=[-4,-2,0,3], index=[2,3,5,10])


full_tex=r"""\begin{array}{  c | c | c | c | c  }
\eta \backslash \xi & -4 & -2 & 0 & 3  \\ \hline
2 & 0.02 & 0.02 & 0.06 & 0.1 \\ \hline
3 & 0.03 & 0.03 & 0.09 & 0.15 \\ \hline
5 & 0.02 & 0.02 & 0.06 & 0.1 \\ \hline
10 & 0.03 & 0.03 & 0.09 & 0.15
\end{array}"""
df = TTDFOVAP(full_tex)

CheckIndependence(df)

True

In [125]:
# Нахождение подбором одного неизвестного значения случайного распределения по известному математическому ожиданию и остальному распределению
for x in range(-1,14):
  vals = list(map(Decimal,f'-1 & 1 & 4 & {x} & 13'.split(' & ')))
  probs  = list(map(Decimal,'0.1 & 0.15 & 0.4 & 0.2 & 0.15'.split(' & ')))
  if myExpectedValue(vals, probs)==Decimal('5.2'): print(x)

8


In [126]:
# Автоматическое нахождение уравнения для математического ожидания, хотя можно было просто посмотреть...
vals='3 5 7'.split(' ')
probs='p1 p2 0.3'.split(' ')
expr=sympy.sympify('+'.join([f'{x1}*{x2}' for x1,x2 in zip(vals,probs)]))
expr

3*p1 + 5*p2 + 2.1

In [127]:
# Тест парсинга в датафрейм + нахождение ожидания произведения 
full_tex=r"""\begin{array}{  c | c | c | c | c  }
\eta \backslash \xi & -4 & -2 & 0 & 3  \\ \hline
2 & 0.02 & 0.02 & 0.06 & 0.1 \\ \hline
3 & 0.03 & 0.03 & 0.09 & 0.15 \\ \hline
5 & 0.02 & 0.02 & 0.06 & 0.1 \\ \hline
10 & 0.03 & 0.03 & 0.09 & 0.15
\end{array}"""
df = TTDFOVAP(full_tex)
TablToExpectedOfMult(df)

Decimal('4.77')

In [128]:
# проверка работы нахождения дисперсии, СКО, 
vals=[0,100]
probs=[99/100,1/100]
MyVariance(vals,probs)

99.0

In [129]:
MyVariance2(vals,probs)

99.0

In [130]:
MyStandardDeviation(vals,probs)

9.9498743710662

In [131]:
# Тест парсинга в vals,probs
full_tex=r"""\begin{array}{  c | c | c | c | c| c | c| c | c  }
\xi & -4 & -2 & 0 & 3 & 4 & 6 & 7 & 9  \\ \hline
\mathsf{P} & 0.1 & 0.15 & 0.05 & 0.15 & 0.1 & 0.05 & 0.25 & 0.15
\end{array}"""
vals,probs=TTDFOVAP(full_tex,dec=0)
vals,probs

(['-4', '-2', '0', '3', '4', '6', '7', '9'],
 ['0.1', '0.15', '0.05', '0.15', '0.1', '0.05', '0.25', '0.15'])

In [148]:
df

,-4,-2,0,3
2,0.02,0.02,0.06,0.1
3,0.03,0.03,0.09,0.15
5,0.02,0.02,0.06,0.1
10,0.03,0.03,0.09,0.15


In [156]:
TablToExpectedOfBoth(df)

(Decimal('0.90'), Decimal('5.30'))